# Second Project



Once syntax trees are built, additional analysis and synthesis can be done by evaluating attributes and executing code fragments on tree nodes. We can also walk through the AST to generate a linear N-address code, analogously to LLVM IR. We call this intermediate machine code as uCIR. So, in this second project, you will perform semantic checks on your program, and turn the AST into uCIR. uCIR uses Single Static Assignment (SSA), and can promote stack allocated scalars to virtual registers and remove the load and store operations, allowing better optimizations since values propagate directly to their use sites.  The main thing that distinguishes SSA from a conventional three-address code is that all assignments in SSA are for distinguished name variables.

## Program Checking
First, you will need to define a symbol table that keeps track of
previously declared identifiers.  The symbol table will be consulted
whenever the compiler needs to lookup information about variable and
constant declarations.

Next, you will need to define objects that represent the different
builtin datatypes and record information about their capabilities.

### Type System
Let's define classes that represent types.  There is a general class used to represent all types.  Each type is then a singleton instance of the type class.
```
class uCType(object):
      pass

int_type = uCType("int",...)
float_type = uCType("float",...)
char_type = uCType("char", ...)
```
The contents of the type class is entirely up to you.  However, you will minimally need to encode some information about what operators are supported (+, -, *, etc.), and default values.

Once you have defined the built-in types, you will need to make sure they get registered with any symbol tables or code that checks for type names.

In [3]:
class uCType(object):
    '''
    Class that represents a type in the uC language.  Types 
    are declared as singleton instances of this type.
    '''
    def __init__(self, name, bin_ops=set(), un_ops=set()):
        '''
        You must implement yourself and figure out what to store.
        '''
        self.name = name
        self.bin_ops = bin_ops
        self.un_ops = un_ops


# Create specific instances of types. You will need to add
# appropriate arguments depending on your definition of uCType
int_type = uCType("int",
    set(('PLUS', 'MINUS', 'TIMES', 'DIVIDE',
         'LE', 'LT', 'EQ', 'NE', 'GT', 'GE')),
    set(('PLUS', 'MINUS')),
    )
float_type = uCType("float",
    set(('PLUS', 'MINUS', 'TIMES', 'DIVIDE',
         'LE', 'LT', 'EQ', 'NE', 'GT', 'GE')),
    set(('PLUS', 'MINUS')),
    )
string_type = uCType("char",
    set(('PLUS',)),
    set(),
    )
boolean_type = uCType("bool",
    set(('AND', 'OR', 'EQ', 'NE')),
    set(('NOT',))
    )
# In your type checking code, you will need to reference the
# above type objects.   Think of how you will want to access
# them.

### Semantic Rules

Finally, you'll need to write code that walks the AST and enforces
a set of semantic rules.  Here is a complete list of everything you'll
need to check:

1.  Names and symbols:

    All identifiers must be defined before they are used.  This includes variables,
    constants, and typenames.  For example, this kind of code generates an error:
```
       a = 3;              // Error. 'a' not defined.
       int a;
```
    Note: typenames such as "int", "float", and "string" are built-in names that
    should be defined at the start of the program.

2.  Types of literals

    All literal symbols must be assigned a type of "int", "float", or "string".  
    For example:
```
       42;         // Type "int"
       4.2;        // Type "float"
       "forty";    // Type "string"
```
    To do this assignment, check the Python type of the literal value and attach
    a type name as appropriate.

3.  Binary operator type checking

    Binary operators only operate on operands of the same type and produce a
    result of the same type.   Otherwise, you get a type error.  For example:
```
        int a = 2;
        float b = 3.14;

        int c = a + 3;    // OK
        int d = a + b;    // Error.  int + float
        int e = b + 4.5;  // Error.  int = float
```

4.  Unary operator type checking.
```
    Unary operators return a result that's the same type as the operand.
```

5.  Supported operators

    Here are the operators supported by each type:
```
    int:      binary { +, -, *, /}, unary { +, -}
    float:    binary { +, -, *, /}, unary { +, -}
    string:   binary { + }, unary { }
```
    Attempts to use unsupported operators should result in an error. 
    For example:
```
        char[] a = "Hello" + "World";     // OK
        char[] b = "Hello" * "World";     // Error (unsupported op *)
```

6.  Assignment.

    The left and right hand sides of an assignment operation must be
    declared as the same type.

    Values can only be assigned to variable declarations, not
    to constants.

For walking the AST, use the NodeVisitor class. A shell of the code is provided below.


In [ ]:
class SymbolTable(object):
    '''
    Class representing a symbol table.  It should provide functionality
    for adding and looking up nodes associated with identifiers.
    '''
    def __init__(self):
        self.symtab = {}
    def lookup(self, a):
        return self.symtab.get(a)
    def add(self, a, v):
        self.symtab[a] = v

class CheckProgramVisitor(NodeVisitor):
    '''
    Program checking class. This class uses the visitor pattern. You need to define methods
    of the form visit_NodeName() for each kind of AST node that you want to process.
    Note: You will need to adjust the names of the AST nodes if you picked different names.
    '''
    def __init__(self):
        # Initialize the symbol table
        self.symtab = SymbolTable()

        # Add built-in type names (int, float, char) to the symbol table
        self.symtab.add("int",uctype.int_type)
        self.symtab.add("float",uctype.float_type)
        self.symtab.add("char",uctype.char_type)
        self.symtab.add("bool",uctype.boolean_type)

    def visit_Program(self,node):
        # 1. Visit all of the statements
        # 2. Record the associated symbol table
        self.visit(node.program)

    def visit_BinaryOp(self, node):
        # 1. Make sure left and right operands have the same type
        # 2. Make sure the operation is supported
        # 3. Assign the result type
        self.visit(node.left)
        self.visit(node.right)
        node.type = node.left.type

    def visit_Assignment(self,node):
        ## 1. Make sure the location of the assignment is defined
        sym = self.symtab.lookup(node.location)
        assert sym, "Assigning to unknown sym"
        ## 2. Check that the types match
        self.visit(node.value)
        assert sym.type == node.value.type, "Type mismatch in assignment"

## Intermediate Representation

At this stage of the project, you are going to turn the AST into an intermediate machine code named uCIR based on Single Static Assignment (SSA). There are a few important parts you'll need to make this work.  Please read 
carefully before beginning:

### Single Static Assignment
The first problem is how to decompose complex expressions into
something that can be handled more simply.  One way to do this is
to decompose all expressions into a sequence of simple assignments
involving binary or unary operations.  

As an example, suppose you had a mathematical expression like this:
```
        2 + 3 * 4 - 5
```
Here is one possible way to decompose the expression into simple
operations:
```
        t_1 = 2
        t_2 = 3
        t_3 = 4
        t_4 = t_2 * t_3
        t_5 = t_1 + t_4
        t_6 = 5
        t_7 = t_5 - t_6
```
In this code, the **t_n** variables are simply temporaries used while
carrying out the calculation.  A critical feature of SSA is that such
temporary variables are only assigned once (single assignment) and
never reused.  Thus, if you were to evaluate another expression, you
would simply keep incrementing the numbers. For example, if you were
to evaluate **10 + 20 + 30**, you would have code like this:
```
        t_8 = 10
        t_9 = 20
        t_10 = t_8 + t_9
        t_11 = 30
        t_12 = t_11 + t_11
```
SSA is meant to mimic the low-level instructions one might carry out 
on a CPU.  For example, the above instructions might be translated to
low-level machine instructions (for a hypothetical RISC-V CPU) like this:

        addi   t1, zero, 2
        addi   t2, zero, 3
        addi   t3, zero, 4
        mul    t4, t2, t3
        addi   t5, t1, t4
        addi   t6, zero, 5
        sub    s1, t5, t6

Another benefit of SSA is that it is very easy to encode and
manipulate using simple data structures such as tuples. For example,
you could encode the above sequence of operations as a list like this:

       [ 
         ('addi', 't1', 0, 2),
         ('addi', 't2', 0, 3),
         ('addi', 't3', 0, 4),
         ('mul', 't4', 't2', 't3'),
         ('addi', 't5', 't1', 't4'),
         ('addi', 't6', 0, 5),
         ('sub', 't7','t5','t6'),
       ]

### Dealing with Variables
In your program, you are probably going to have some variables that get
used and assigned different values.  For example:
```
       a = 10 + 20;
       b = 2 * a;
       a = a + 1;
```
In "pure SSA", all of your variables would actually be versioned just
like temporaries in the expressions above.  For example, you would
emit code like this:
```
       t_1 = 10
       t_2 = 20
       a_1 = t_1 + t_2
       t_3 = 2
       b_1 = t_3 * a_1
       t_4 = 1 
       a_2 = a_1 + t_4
       ...
```
To avoid this, we're going to treat declared variables as memory locations and access them using load/store
instructions.  For example:
```
       t_1 = 10
       t_2 = 20
       t_3 = t_1 + t_2
       store(t_3, "a")
       t_4 = 2
       t_5 = load("a")
       t_6 = t_4 * t_5
       store(t_6,"b")
       t_7 = load("a")
       t_8 = 1
       t_9 = t_7 + t_8
       store(t_9, "a")
```

### A Word About Types
At a low-level, CPUs can only operate a few different kinds of 
data such as ints and floats.  Because the semantics of the
low-level types might vary slightly, you'll need to take 
some steps to handle them separately.

In our intermediate code, we're simply going to tag temporary variable
names and instructions with an associated type low-level type.  For
example:

      2 + 3 * 4          (ints)
      2.0 + 3.0 * 4.0    (floats)

The generated intermediate code might look like this:

      ('literal_int', 2, 't_1')
      ('literal_int', 3, 't_2')
      ('literal_int', 4, 't_3')
      ('mul_int', 't_2', 't_3', 't_4')
      ('add_int', 't_1', 't_4', 't_5')

      ('literal_float', 2.0, 't_6')
      ('literal_float', 3.0, 't_7')
      ('literal_float', 4.0, 't_8')
      ('mul_float', 't_7', 't_8', 't_9')
      ('add_float', 't_6', 't_9', 't_10')

### Your Task
Your task is as follows: Write a AST Visitor() class that takes an
uC program and flattens it to a single sequence of SSA code instructions
represented as tuples of the form 
```
       (operation, operands, ..., destination)
```
Your SSA code should only contain the following operators:

#### Variables & Values:
```
       ('alloc_type', varname)          # Allocate on stack (ref by register) a variable of a given type.
       ('global_type', varname, value)  # Allocate on heap a global var of a given type. value is optional.
       ('load_type', varname, target)   # Load the value of a variable (stack or heap) into target (register).
       ('store_type', source, target)   # Store the source/register into target/varname.
       ('literal_type', value, target)  # Load a literal value into target.
```

#### Binary Operations:
```
       ('add_type', left, right, target )  # target = left + right
       ('sub_type', left, right, target)   # target = left - right
       ('mul_type', left, right, target)   # target = left * right
       ('div_type', left, right, target)   # target = left / right  (integer truncation)
       ('mod_type', left, right, target)   # target = left % rigth
```

#### Unary Operations:
```
       ('uadd_type', source, target)        # target = +source
       ('uneg_type', source, target)        # target = -source
```

#### Relational/Equality/Logical:
```
       (`oper`, left, right, target)   # target = left `oper` rigth, where `oper` is:
                                                  lt, le, ge, gt, eq, ne, and, or
```

#### Labels & Branches:
```
       ('label', )                                       # Label definition
       ('jump', target)                                  # Jump to a target label
       ('cbranch, expr_test, true_target, false_target)  # Conditional Branch
```

#### Functions & Builtins:
```
       ('define', source)               # Function definition. Source is a function label 
       ('end', )                        # End of a Function definition
       ('call', source, target)         # Call a function. target is an optional return value
       ('return_type', source, target)  # Return from function. target is an optional return value
       ('param_type', source)           # source is an actual parameter
       ('read_type', source)            # Read value to source
       ('print_type',source)            # Print value of source
```

### uCIR Examples

```
int n = 10;

int foo(int a, int b) {
    return n * (a + b);
}

int main() {
    int c = 2, d = 3;
    int e = foo(c, d);
    return 0;
}

('global_int', '@n', 10)
('define' , '_foo:')
; function arguments: "a" is referenced by t_0 ; "b" by t_1 & return slot (on stack) by t_2
('alloca_int', 't_3')
('alloca_int', 't_4')
('store_int', 't_0', 't_3')
('store_int', 't_1', 't_4')
('load_int', '@n', 't_5')
('load_int', 't_3', 't_6')
('load_int', 't_4', 't_7')
('add_int', 't_6', 't_7', 't_8')
('mul_int', 't_5', 't_8', 't_9')
('return_int', 't_9', 't_2')
('end')

('define', '_main:')
; function arguments: return slot (on stack) is referenced by t_0
('alloca_int', 't_1')
('alloca_int', 't_2')
('alloca_int', 't_3')
('literal_int', 2, 't_1')
('literal_int', 3, 't_2')
('load_int', 't_1', 't_4')
('load_int', 't_2', 't_5')
('param_int', 't_4')
('param_int', 't_5')
('call', '_foo:', 't_6')
('store_int', 't_6', 't_3')
('return_int', 0, 't_0')
('end')
```

# Generating Code

Implement the following Node Visitor class so that it creates
a sequence of SSA instructions in the form of tuples.  Use the
above description of the allowed op-codes as a guide.

In [ ]:
class GenerateCode(exprast.NodeVisitor):
    '''
    Node visitor class that creates 3-address encoded instruction sequences.
    '''
    def __init__(self):
        super(GenerateCode, self).__init__()

        # version dictionary for temporaries
        self.versions = defaultdict(int)

        # The generated code (list of tuples)
        self.code = []

    def new_temp(self,typeobj):
        '''
        Create a new temporary variable of a given type.
        '''
        name = "t_%d" % (self.versions[typeobj.name])
        self.versions[typeobj.name] += 1
        return name

    # You must implement visit_Nodename methods for all of the other
    # AST nodes.  In your code, you will need to make instructions
    # and append them to the self.code list.
    #
    # A few sample methods follow.  You may have to adjust depending
    # on the names of the AST nodes you've defined.

    def visit_Literal(self,node):
        # Create a new temporary variable name 
        target = self.new_temp(node.type)

        # Make the SSA opcode and append to list of generated instructions
        inst = ('literal_'+node.type.name, node.value, target)
        self.code.append(inst)

        # Save the name of the temporary variable where the value was placed 
        node.gen_location = target

    def visit_BinaryOp(self,node):
        # Visit the left and right expressions
        self.visit(node.left)
        self.visit(node.right)

        # Make a new temporary for storing the result
        target = self.new_temp(node.type)

        # Create the opcode and append to list
        opcode = binary_ops[node.op] + "_"+node.left.type.name
        inst = (opcode, node.left.gen_location, node.right.gen_location, target)
        self.code.append(inst)

        # Store location of the result on the node
        node.gen_location = target

    def visit_PrintStatement(self,node):
        # Visit the expression
        self.visit(node.expr)

        # Create the opcode and append to list
        inst = ('print_'+node.expr.type.name, node.expr.gen_location)
        self.code.append(inst)

    def visit_VarDeclaration(self,node):
        # allocate on stack memory
        inst = ('alloc_'+node.type.name, 
                    node.id)
        self.code.append(inst)
        # store optional init val
        if node.value:
            self.visit(node.value)
            inst = ('store_'+node.type.name,
                    node.value.gen_location,
                    node.id)
            self.code.append(inst)

    def visit_LoadLocation(self,node):
        target = self.new_temp(node.type)
        inst = ('load_'+node.type.name,
                node.name,
                target)
        self.code.append(inst)
        node.gen_location = target

    def visit_AssignmentStatement(self,node):
        self.visit(node.value)
        inst = ('store_'+node.value.type.name, 
                node.value.gen_location, 
                node.location)
        self.code.append(inst)

    def visit_UnaryOp(self,node):
        self.visit(node.left)
        target = self.new_temp(node.type)
        opcode = unary_ops[node.op] + "_" + node.left.type.name
        inst = (opcode, node.left.gen_location)
        self.code.append(inst)
        node.gen_location = target

# Writing an Interpreter

Once you've got your compiler emitting intermediate code, you should
be able to write a simple interpreter that runs the code.  This
can be useful for prototyping the execution environment, testing,
and other tasks involving the generated code.

Your task is simple, extend the Interpreter class below so that it
can run the code you generated above.  The comments and docstrings
in the class describe it in further details.

In [8]:
class Interpreter(object):
    '''
    Runs an interpreter on the SSA intermediate code generated for
    your compiler.   The implementation idea is as follows.  Given
    a sequence of instruction tuples such as:

         code = [ 
              ('literal_int', 1, 't_1'),
              ('literal_int', 2, 't_2'),
              ('add_int', 't_1', 't_2, 't_3')
              ('print_int', 't_3')
              ...
         ]

    The class executes methods self.run_opcode(args).  For example:

             self.run_literal_int(1, 't_1')
             self.run_literal_int(2, 't_2')
             self.run_add_int('t_1', 't_2', 't_3')
             self.run_print_int('t_3')

    To store the values of variables created in the intermediate
    language, simply use a dictionary.

    For builtin function declarations, allow specific Python modules
    (e.g., print, input, etc.) to be registered with the interpreter.
    We don't have namespaces in the source language so this is going
    to be a bit of sick hack.
    '''
    def __init__(self,name="module"):
        # Dictionary of currently defined variables
        self.vars = {}

    def run(self, ircode):
        '''
        Run intermediate code in the interpreter.  ircode is a list
        of instruction tuples.  Each instruction (opcode, *args) is 
        dispatched to a method self.run_opcode(*args)
        '''
        self.pc = 0
        while True:
            try:
                op = ircode[self.pc]
            except IndexError:
                if self.pc > len(ircode):
                    print("Wrong PC %d - terminating" % self.pc)
                return
            self.pc += 1
            opcode = op[0]
            if hasattr(self, "run_"+opcode):
                getattr(self, "run_"+opcode)(*op[1:])
            else:
                print("Warning: No run_"+opcode+"() method")
        
    # YOU MUST IMPLEMENT methods for different opcodes.  A few sample
    # opcodes are shown below to get you started.

    def run_jump(self, label):
        self.pc = label

    def run_cbranch(self, cond, if_label, else_label):
        if self.vars[cond]:
            self.pc = if_label
        else:
            self.pc = else_label

    def run_literal_int(self, value, target):
        '''
        Create a literal integer value
        '''
        self.vars[target] = value

    run_literal_float = run_literal_int
    run_literal_char = run_literal_int
    
    def run_add_int(self, left, right, target):
        '''
        Add two integer variables
        '''
        self.vars[target] = self.vars[left] + self.vars[right]

    run_add_float = run_add_int
    run_add_string = run_add_int

    def run_print_int(self, source):
        '''
        Output an integer value.
        '''
        print(self.vars[source])

    def run_alloc_int(self, name):
        self.vars[name] = 0

    def run_alloc_float(self, name):
        self.vars[name] = 0.0

    def run_alloc_char(self, name):
        self.vars[name] = ''

    def run_store_int(self, source, target):
        self.vars[target] = self.vars[source]

    run_store_float = run_store_int
    run_store_char = run_store_int

    def run_load_int(self, name, target):
        self.vars[target] = self.vars[name]

    run_load_float = run_load_int
    run_load_char = run_load_int

    def run_sub_int(self, left, right, target):
        self.vars[target] = self.vars[left] - self.vars[right]

    run_sub_float = run_sub_int

    def run_mul_int(self, left, right, target):
        self.vars[target] = self.vars[left] * self.vars[right]

    run_mul_float = run_mul_int

    def run_div_int(self, left, right, target):
        self.vars[target] = self.vars[left] // self.vars[right]

    def run_div_float(self, left, right, target):
        self.vars[target] = self.vars[left] / self.vars[right]

    def run_cmp_int(self, op, left, right, target):
        compare = cmp(self.vars[left], self.vars[right])
        if op == 'lt':
            result = bool(compare < 0)
        elif op == 'le':
            result = bool(compare <= 0)
        elif op == 'eq':
            result = bool(compare == 0)
        elif op == 'ne':
            result = bool(compare != 0)
        elif op == 'ge':
            result = bool(compare >= 0)
        elif op == 'gt':
            result = bool(compare > 0)
        elif op == 'land':
            result = self.vars[left] and self.vars[right]
        elif op == 'lor':
            result = self.vars[left] or self.vars[right]
        self.vars[target] = result

    run_cmp_float = run_cmp_int
    run_cmp_bool = run_cmp_int

    run_print_float = run_print_int
    run_print_char = run_print_int

    def run_call(self, funcname, *args):
        '''
        Call a previously declared function.
        '''
        target = args[-1]
        func = self.vars.get(funcname)
        argvals = [self.vars[name] for name in args[:-1]]
        self.vars[target] = func(*argvals)